In [1]:
import os
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers, Model
import pathlib
from tensorflow.keras import Input
import numpy as np
from tensorflow.keras.layers import Lambda, Conv2D, BatchNormalization, MaxPooling2D, Conv2DTranspose, concatenate
from tensorflow.keras.metrics import MeanIoU

2022-12-01 15:56:01.816959: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-01 15:56:01.817007: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
#Deviding data by batches

In [3]:
path_X = '/home/lewagon/code/Dponomareva/test/data/Images'

In [4]:
path_y = '/home/lewagon/code/Dponomareva/test/data/Target'

In [5]:
split_ratio = 0.8

In [6]:
def train_val_split (path_X, path_y, split_ratio):
    X_names = os.listdir(path_X)
    y_names = os.listdir(path_y)
    y_path = [f'{path_y}/{file}' for file in y_names]
    X_path = [f'{path_X}/{file}' for file in X_names]
    train_X, val_X = X_path[:int(len(X_path)*split_ratio)], X_path[int(len(X_path)*split_ratio):]
    train_y, val_y = y_path[:int(len(y_path)*split_ratio)], y_path[int(len(y_path)*split_ratio):]
    return train_X, val_X, train_y, val_y 

In [7]:
train_X, val_X, train_y, val_y = train_val_split (path_X, path_y, split_ratio)

In [8]:
def verify_matching_input_labels(X_names, y_names):
    for x, y in zip(X_names, y_names):
        if os.path.basename(x) != os.path.basename(y):
            raise ValueError(f"X and Y not matching: {x, y}")

In [9]:
verify_matching_input_labels(train_X, train_y)

In [10]:
verify_matching_input_labels(val_X, val_y)

In [11]:
import keras.backend as K

In [12]:
def process_path(image_path, mask_path):
    image = tf.io.read_file(image_path)
    mask = tf.io.read_file(mask_path)
    image = tf.image.decode_png(image, channels = 3)
    mask = tf.image.decode_png(mask, channels = 1)
    return image, mask

In [24]:
def batch_data (path_X, path_y, batch_size):
    ds_train = tf.data.Dataset.from_tensor_slices((path_X, path_y))
    return ds_train.map(process_path).batch(batch_size)
    #return ds_train.shuffle(buffer_size = len(path_X), seed = 10).map(process_path).batch(batch_size)

In [14]:
def loss_sum(y_true, y_pred):
    def dice_loss(targets, inputs, smooth=1e-6):
    #flatten label and prediction tensors
        inputs = K.flatten(inputs)
        targets = K.flatten(targets)
    
        intersection = K.sum(targets * inputs)
        dice = (2*intersection + smooth) / (K.sum(targets) + K.sum(inputs) + smooth)
        return 1 - dice

    y_true = tf.cast(y_true, tf.float32)
    #y_pred = tf.cast(y_pred, tf.float32)
    o = tf.keras.losses.BinaryCrossentropy()(y_true, y_pred) + dice_loss(y_true, y_pred)
    return tf.reduce_mean(o)

In [15]:
def unet_model(img_height, img_width, channels):

    #Input
    inputs = Input((img_height, img_width, channels))
    inputs = Lambda(lambda x: x / 255)(inputs) #Normalize the pixels by dividing by 255

    #Encoder where we are extracting the features
    convolution1 = Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same')(inputs)
    convolution1 = BatchNormalization()(convolution1) #other option is to do dropout by batch is faster 
    convolution1 = Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same')(convolution1)
    pooling1 = MaxPooling2D((2, 2))(convolution1)
      
    convolution1 = Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same')(inputs)
    convolution1 = BatchNormalization()(convolution1) 
    convolution1 = Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same')(convolution1)
    pooling1 = MaxPooling2D((2, 2))(convolution1)
    
    convolution2 = Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')(pooling1)
    convolution2 = BatchNormalization()(convolution2)
    convolution2 = Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')(convolution2)
    pooling2 = MaxPooling2D((2, 2))(convolution2)
     
    convolution3 = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(pooling2)
    convolution3 = BatchNormalization()(convolution3)
    convolution3 = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(convolution3)
    pooling3 = MaxPooling2D((2, 2))(convolution3)
     
    convolution4 = Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same')(pooling3)
    convolution4 = BatchNormalization()(convolution4)
    convolution4 = Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same')(convolution4)
    pooling4 = MaxPooling2D(pool_size=(2, 2))(convolution4)

    #Bottleneck at the base of the U-net 
    convolution5 = Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same')(pooling4)
    convolution5 = BatchNormalization()(convolution5)
    convolution5 = Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same')(convolution5)
    
    #Decoder where we are indicating to the model the precise location of the features 
    transconv6 = Conv2DTranspose(128, kernel_size=(2, 2), strides=(2, 2), padding='same')(convolution5)
    transconv6 = concatenate([transconv6, convolution4])
    convolution6 = Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same')(transconv6)
    convolution6 = BatchNormalization()(convolution6)
    convolution6 = Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same')(convolution6)
     
    transconv7 = Conv2DTranspose(64, kernel_size=(2, 2), strides=(2, 2), padding='same')(convolution6)
    transconv7 = concatenate([transconv7, convolution3])
    convolution7 = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(transconv7)
    convolution7 = BatchNormalization()(convolution7)
    convolution7 = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(convolution7)
     
    transconv8 = Conv2DTranspose(32, kernel_size=(2, 2), strides=(2, 2), padding='same')(convolution7)
    transconv8 = concatenate([transconv8, convolution2])
    convolution8 = Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')(transconv8)
    convolution8 = BatchNormalization()(convolution8)
    convolution8 = Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')(convolution8)
     
    transconv9 = Conv2DTranspose(16, kernel_size=(2, 2), strides=(2, 2), padding='same')(convolution8)
    transconv9 = concatenate([transconv9, convolution1], axis=3)
    convolution9 = Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same')(transconv9)
    convolution9 = BatchNormalization()(convolution9)
    convolution9 = Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same')(convolution9)
     
    outputs = Conv2D(1, kernel_size=(1, 1), activation='sigmoid')(convolution9)
     
    model = Model(inputs=[inputs], outputs=[outputs])
    jaccard = MeanIoU(2)
    
    #loss options include: binary_crossentropy, IoU Loss (Jaccard Index), dice loss
    model.compile(optimizer='adam', loss=loss_sum) 
    
    model.summary()
    
    return model

In [16]:
channels=3

In [17]:
img_height = 256

In [18]:
img_width = 256

In [19]:
model = unet_model(img_height, img_width, channels)

2022-12-01 15:56:04.678192: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-01 15:56:04.678273: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-01 15:56:04.678307: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (SLB-G9SSST2): /proc/driver/nvidia/version does not exist
2022-12-01 15:56:04.678541: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_2 (Conv2D)              (None, 256, 256, 16  448         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_1 (BatchNo  (None, 256, 256, 16  64         ['conv2d_2[1][0]']               
 rmalization)                   )                                                             

In [20]:
#from tensorflow.keras.utils import plot_model

In [21]:
#plot_model(model,
#    to_file='model.png',
#    show_shapes=True, show_dtype=True,
#    show_layer_names=True,
#    rankdir='TB',
#    expand_nested=True,
#    dpi=96,
#    layer_range=None,
#    show_layer_activations=True)

In [22]:
batch_size = 32

In [23]:
model.fit(batch_data(train_X, train_y,batch_size), epochs=1, validation_data=batch_data(val_X, val_y,batch_size))

TypeError: in user code:

    File "/home/lewagon/.pyenv/versions/3.8.12/envs/solar/lib/python3.8/site-packages/keras/engine/training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "/tmp/ipykernel_3525/1777002939.py", line 7, in dice_loss  *
        intersection = K.sum(targets * inputs)

    TypeError: Input 'y' of 'Mul' Op has type float32 that does not match type uint8 of argument 'x'.
